# Numerical simulation of the wave equation in 1D using Finite Difference

## Import libraries

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as anim
from IPython.display import HTML
import sys

----

## Create domain model properties (i.e. speed of waves), and discretisation grid

In [ ]:
nx = 801  # 801 sample-points along the x-axis

# start with homogeneous model - same wave-speed everywhere.
c = np.full(nx,1500.0)  # Note: 1500m/s is typical acoustic velocity of water

# a faster section, halfway across model, to show reflection of waves when crossing sharp interface
c[400:] = 2500.0  # Note: 2500m/s is fairly typical acoustic velocity of sedimentary rock

In [ ]:
length = 6000.0  # assign a length to the model in metres

dx = length/(nx-1)  # calculate the spatial increment between model parameters

print('Domain is %.1fm (%d sample-points)' % (length,nx))
print('Sample-point spacing (dx) is %.5fm' % (dx))

In [ ]:
plt.figure(figsize=(10,6))
plt.plot(c) # plotting the velocity model
plt.xlabel('x gridpoints')
plt.ylabel('speed')
plt.title('Velocity Model (m/s)')
plt.show()

----
## Modelling constraints
#### CFL stability condition should be satisfied – the 'Courant number'.

The dimensionless Courant number, $C$, gives a measure of how far a signal will travel between grid-points within one time-step.
(Hence why I will very often refer to it as 'the crossing factor'.)

The crossing factor is $c$.$\delta t$/$\delta x$ for speed $c$, and we want the maximum value of this within the whole domain to satisfy some constraint that depends upon the finite-difference stencil(s) in use when modelling.

i.e. we want:$\quad \text{max}(c)$.$\delta t/\delta x <= C_{max}$, where $\ C_{max}$ depends on the discretisation.

We can turn this around to find the maximum time-step for our model and grid-spacing, given $C_{max}$ for our discretisation:

$$\delta t_{best} = \frac{C_{max}\ \delta x}{\text{max}(c)}$$

#### This simple stencil (2nd order), with 2nd order time-stepping, for the 1d wave equation, allows max crossing factor to be up to 1.0
(i.e. can only cross up to 100% of a cell in one time-step – but we'll use 80% below.)

In [ ]:
C = 0.8  # set the dimensionless max Courant number that we want to use to work out time-step below

In [ ]:
time = 5.0  # desired length of the simulation in seconds
dt = (C*dx)/c.max()  # define the best time-step by using the max Courant number
nt = int(time/dt+0.9999)  # find how many time-steps are needed to (very very nearly) cover that time
time = nt*dt  # turn that back into exact time for this number of steps

print('Time-step = %.5fs  Number of steps = %d  (Total time being modelled: %.5fs)' % (dt,nt,dt*nt))

sx = 150  # position for source point - at 151st sample-point


### Create source function based on sine-squared curve...

In [ ]:
def create_sinesq_src(amp,ns):  # ns is number of steps for our source function

    src = np.zeros(ns)  # create array for source function

    # create source function with sine-squared curve (which starts off zero, rises to one, then back to zero)

    for i in range(ns):
        src[i] = amp*np.sin((0.5+i)*np.pi/ns)**2

    print('Start & end values are: %.6f %.6f' % (src[0],src[-1]))
    return src

In [ ]:
def plot_source(src):
    plt.figure(figsize=(10,6))
    plt.plot(src) # plot source function
    plt.xlabel('timesteps')
    plt.ylabel('amplitude')
    plt.title('Source function')
    plt.show()

In [ ]:
# want to locate the source at some point in the domain, and give it an amplitude

ns = 40   # make source function last for 40 time-steps (will also use this later)
src = create_sinesq_src(1.0,ns)

plot_source(src)

### Check that the maximum frequency in the source function can propagate reliably

For a simple second-order finite-difference, the minimum wavelength of a signal that we can propagate reliably over a reasonable distance is about 10 grid-points.

In [ ]:
# minimum of 10 gridpoints per wavelength for reasonably accurate propagation (i.e. minimal numerical dispersion):
min_cells_per_wl = 10.0

# calculate the max frequency that can be modelled without numerical dispersion:
max_freq = c.min()/(min_cells_per_wl*dx)

print('Maximum reliable propagation frequency is about %.1fHz' % max_freq)

#### Want to avoid causing too much dispersion by keeping maximum significant frequency within this limit...

In [ ]:
def plot_ampl_spect(src,ns,nt,dt):
    # plot amplitude spectrum of source function
    plt.figure(figsize=(10,6))
    plt.magnitude_spectrum(np.append(src,np.zeros(nt-ns)), Fs=1/dt)  # note padding to nt points
    plt.title('Amplitude Spectrum')
    plt.xlim(0,35)
    plt.xlabel('Frequency / Hz')
    plt.show()

In [ ]:
plot_ampl_spect(src,ns,nt,dt)

----
# Simulation

In [ ]:
# prepare an array to store wavefield snapshots for plotting
snapshot_gap = 5 # set sampling rate used to store wavefield (every 10 timesteps)
wavefield = np.zeros((int(nt/snapshot_gap), nx)) # array to store wavefields every 10 timesteps
print('Storing %d wavefields (every %dth out of %d)' % (wavefield.shape[0],snapshot_gap,nt))

In [ ]:
# Initialise arrays for wavefields
u = np.zeros(nx) # current wavefield
u_prv = np.zeros(nx) # previous wavefield, at t-1
u_nxt = np.zeros(nx) # next wavefield, at t+1

# a useful variable – shorthand for terms that appear together in equations
dtdx2 = (dt*dt)/(dx*dx)

In [ ]:
def propagate():

    # make sure wavefields start off zero before propagating (source gets included during steps)
    u[:] = 0.0
    u_prv[:] = 0.0

    u[sx] = src[0]  # put first source entry into current wavefield

    # begin time-stepping loop...

    for i in range(nt):

        if i%20==0:  # show progress every 20 steps
            sys.stdout.write('Done step %d of %d\r' % (i+1,nt))

        # find new wavefield, u_new, throughout domain (apart from edges)
        # NOTE: no vectorisation, so it's slow!
        for ix in range(1,nx-1):
            u_nxt[ix] = (c[ix]**2) * dtdx2 * (u[ix-1]-2*u[ix]+u[ix+1]) + 2*u[ix] - u_prv[ix]

        # put in source function as if it's a 'forced' condition (rather than injecting by adding),
        # up until the source function runs out of its steps
        if i+1<ns:
            u_nxt[sx] = src[i+1]
    
    
        # FILL IN CODE HERE TO RECORD VALUES AT DETECTOR POINTS, using arrays rpos & r (see task 2)
    
    
    
        # cycle wavefields for next time-step
        u_prv[:] = u[:]
        u[:] = u_nxt[:]
    
        # store the current wavefield u on every Nth step (where N is snapshot_gap, defined earlier)
        if (i+1)%snapshot_gap == 0:
            wavefield[int((i+1)/snapshot_gap-1),:] = u[:]

    print('Finished all %d steps' % (nt))

In [ ]:
propagate()

## Plot wavefield time snapshots

In [ ]:
def plot_snapshot(plot_time):
    plt.figure(figsize=(15,5))
    plt.plot(wavefield[int(plot_time/(dt*snapshot_gap)),:])
    plt.title('Wavefield at about %.2fs' % (plot_time))
    plt.xlabel('x gridpoints')
    plt.ylabel('amplitude')
    plt.show()

In [ ]:
plot_snapshot(1.5)
plot_snapshot(4.2)

## Show space-time plot for whole wavefield

In [ ]:
def spacetime_plot():
    fig = plt.figure(figsize=(10,8))
    plt.imshow(wavefield, cmap='RdBu', interpolation='bilinear', aspect='auto',
               vmin=-1, vmax=1,       # set the bounds for the colour data
               extent=(0,length,time,0))  # set the bounds for the axes
    plt.title('Wavefield propagation with time')
    plt.xlabel('x-position / m')
    plt.ylabel('Time / s')
    plt.show()


In [ ]:
spacetime_plot()

## Make a movie! 

In [ ]:
def create_animation():

    fig, ax = plt.subplots(figsize=(15,6))

    x = np.arange(0, wavefield.shape[1], 1)
    line, = ax.plot(x,wavefield[0])

    plt.title('Wavefield')
    plt.xlabel('x gridpoint')
    plt.ylabel('amplitude')
    plt.ylim(-1.3,1.3)

    def frame(i):
        line.set_ydata(wavefield[i])
        return line,

    print('Finished plots for frames, building animation...')

    ani = anim.FuncAnimation(fig, frame, interval=50, save_count=wavefield.shape[0])

    plt.close(fig)  # prevent final image from showing up inline below
    return ani


In [ ]:
ani = create_animation()
print('Preparing HTML (takes a little while...)')
HTML(ani.to_jshtml())

----
## Create source function based on Ricker wavelet with peak frequency of 6Hz

The Ricker wavelet used here is a function of time that's defined from the second derivate of a Gaussian function (which is $G(t)=\text{e}^{-a^2 t^2}$).

–It's also known as the Mexican hat wavelet, due to its shape when plotted as a function of two variables.

It can be written so it is symmetric about time zero, with maximum at that time, ultimately decaying towards zero as $t$$\rightarrow$$±\infty$. However, we will shift it in time so that it starts near zero at our time zero.

After differentiating above Gaussian, $G(t)$, twice, flipping, then scaling (and before shifting in time), we get: $\quad R(t) = (1-2a^2 t^2)\ \text{e}^{-a^2 t^2}$

The peak frequency of the Ricker wavelet is at $f$=$a/\pi$.

The length of the wavelet, in time, before it decays close enough to zero for our purposes, is about $\frac{2.1}{f}$ (i.e. about $\frac{1.05}{f}$ seconds each side of the wavelet's central peak).

>**Task 1:**
>
>**Fill in code below to fill array `src` with a source function based on Ricker wavelet (see equations above)**

In [ ]:
def ricker_source(freq,dt,ampl):
    ts = 2.1/freq  # the length in time is related to frequency
    ns = int(ts/dt+0.9999)  # figure out how many time-steps are needed to cover that time
    src = np.zeros(ns)  # create array with new length, for wavelet
    print('Length of source is %d steps (%.5f secs, was %.5fs)' % (ns,ns*dt,ts))
    ts = ns*dt  # and now turn that back into a time that's exactly the required number of steps
    a2 = (freq*np.pi)**2  # a squared (see equation above)
    t0 = ts/2 - dt/2  # midpoint time of wavelet


    # FILL IN CODE HERE TO CREATE A RICKER WAVELET SOURCE FUNCTION (see equation above)...
    # Its mid-point should be at time t0, which comes from code lines above



    print('Endpoint values are: %.6f %.6f' % (src[0],src[-1]))
    return src,ns

In [ ]:
src,ns = ricker_source(6.0,dt,1.0)  # want Ricker wavelet to have peak frequency of 6Hz
plot_source(src)

#### Check that the maximum frequency in the RIcker source function can propagate reliably...
Also note that the frequency spectrum shows the peak frequency at 6Hz, just as we wanted above.

In [ ]:
plot_ampl_spect(src,ns,nt,dt)
print('Maximum reliable propagation frequency is about %.1fHz' % max_freq)

## Receiver points within domain
–To detect what crosses a few particular points of the domain over time

>**Task 2:**
>
>**Create an array named `rpos` which will contain the positions of the detector points.**
>
>**Place detectors at three sample-point positions: 70, 100 & 640.**
>
>**Create a 2d array named `r` which will be used to record the wavefield at each detector and each time-step.**  
(The first dimension should be the same as number of detector points, second should be number of time-steps.)
>
>**Go to the time-stepping propagation loop again, and fill in code (where indicated) to place the values of the current wavefield into the appropriate elements of the detector array `r` (based on the gridpoint positions given in array `rpos`)**
>


In [ ]:
# FILL IN CODE HERE TO INITIALISE ARRAYS rpos & r...




In [ ]:
propagate()

In [ ]:
# plot a couple of snapshots
plot_snapshot(1.5)
plot_snapshot(4.2)

### Can now plot data at receivers (detector points)...

In [ ]:
def plot_at_receiver(i):
    plt.figure(figsize=(15,5))
    plt.plot(r[i])
    plt.title('Data for receiver at grid-point %d' % (rpos[i]))
    plt.ylabel('Amplitude')
    plt.xlabel('Time-steps')
    plt.show()

In [ ]:
for i in range(r.shape[0]):
    plot_at_receiver(i)

In [ ]:
spacetime_plot()

In [ ]:
ani = create_animation()
print('Preparing HTML (takes a little while...)')
HTML(ani.to_jshtml())